In [1]:
bucket = 'bucket-mldemo-dev-20200327141354666900000003'
 
# Define IAM role
import re


In [2]:
import boto3
boto3_session=boto3.Session(profile_name='ml-lab')

In [3]:
from sagemaker.session import Session
from sagemaker import get_execution_role

sagemaker_session=Session(boto3_session)

try:
    role = get_execution_role()
except:
    iam=boto3_session.client('iam')
    response=iam.get_role(
        RoleName='mldemo-dev-Sagemaker-Execution-Role'
    )
    role=response['Role']['Arn']
role

'arn:aws:iam::814948925568:role/mldemo-dev-Sagemaker-Execution-Role'

In [5]:
output_location = 's3://{}/output'.format(bucket)
output_location

's3://bucket-mldemo-dev-20200327141354666900000003/output'

In [6]:
from sagemaker.predictor import RealTimePredictor
linear_predictor=RealTimePredictor('mldemo-dev',sagemaker_session)

In [7]:
from sagemaker.predictor import csv_serializer, json_deserializer

linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

In [8]:
from s3fs.core import S3FileSystem
s3fs = S3FileSystem(anon=False,session=boto3_session)

import pandas as pd
df=pd.read_csv(s3fs.open('s3://bucket-mldemo-dev-20200327141354666900000003/transformed/all_data.csv'),header=None)
# evaluation_data_df.loc[0].tolist()[0:72]
df

,0,1,2,3,4,5,6,7,8,9,...,596,597,598,599,600,601,602,603,604,605
0,-18.0,18.0,357.70,393.90,-97.30,0.000000,-0.170000,0.000000,-0.170,-0.100,...,-28.8500,-29.0550,-29.1500,-27.60,-2.746000e+01,-27.630,-27.8000,-27.850000,-27.800000,0
1,-18.0,18.0,357.87,394.07,-97.13,0.170000,0.000000,0.170000,0.000,0.070,...,-28.6800,-28.8850,-28.9800,-27.43,-2.729000e+01,-27.460,-27.6300,-27.680000,-27.630000,0
2,-18.0,18.0,357.70,393.90,-97.30,0.000000,-0.170000,0.000000,-0.170,-0.100,...,-28.8500,-29.0550,-29.1500,-27.60,-2.746000e+01,-27.630,-27.8000,-27.850000,-27.800000,0
3,-18.0,18.0,357.87,394.07,-97.13,0.170000,0.000000,0.170000,0.000,0.070,...,-28.6800,-28.8850,-28.9800,-27.43,-2.729000e+01,-27.460,-27.6300,-27.680000,-27.630000,0
4,-18.0,18.0,357.80,394.00,-97.20,0.100000,-0.070000,0.100000,-0.070,0.000,...,-28.7500,-28.9550,-29.0500,-27.50,-2.736000e+01,-27.530,-27.7000,-27.750000,-27.700000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10002,-18.0,18.0,433.22,469.42,-73.48,-12.270000,-12.270000,-11.936667,-11.770,-12.070,...,-2.4900,-2.4625,-2.6125,-2.35,0.000000e+00,0.050,0.1000,-0.125000,-0.062500,0
10003,-18.0,18.0,433.17,469.37,-73.53,-12.320000,-11.986667,-11.820000,-12.120,-12.855,...,-2.5125,-2.6625,-2.4000,-0.05,-9.094947e-13,0.050,-0.1750,-0.112500,-0.166667,0
10004,-18.0,18.0,433.22,469.42,-73.48,-12.270000,-11.936667,-11.770000,-12.070,-12.805,...,-2.4625,-2.6125,-2.3500,0.00,5.000000e-02,0.100,-0.1250,-0.062500,-0.116667,0
10005,-18.0,18.0,433.27,469.47,-73.43,-12.220000,-11.886667,-11.720000,-12.020,-12.755,...,-2.4125,-2.5625,-2.3000,0.05,1.000000e-01,0.150,-0.0750,-0.012500,-0.066667,0


In [9]:
evaluation_data_df=df[round(len(df)*0.85+1):].copy().reset_index()
evaluation_data_df

,index,0,1,2,3,4,5,6,7,8,...,596,597,598,599,600,601,602,603,604,605
0,8507,-18.0,18.0,422.10,458.30,-84.60,32.086667,32.063333,31.815000,31.840,...,0.9400,0.4200,0.0800,-0.77,-1.280000e+00,-0.770,0.6100,0.510000,0.340000,0
1,8508,-18.0,18.0,422.71,458.91,-83.99,32.673333,32.425000,32.450000,32.560,...,1.0300,0.6900,-0.1600,-0.67,-1.600000e-01,1.220,1.1200,0.950000,0.610000,0
2,8509,-18.0,18.0,422.76,458.96,-83.94,32.475000,32.500000,32.610000,32.460,...,0.7400,-0.1100,-0.6200,-0.11,1.270000e+00,1.170,1.0000,0.660000,0.050000,0
3,8510,-18.0,18.0,421.65,457.85,-85.05,31.390000,31.500000,31.350000,31.250,...,-1.2200,-1.7300,-1.2200,0.16,6.000000e-02,-0.110,-0.4500,-1.060000,-1.110000,0
4,8511,-18.0,18.0,422.34,458.54,-84.36,32.190000,32.040000,31.940000,32.470,...,-1.0400,-0.5300,0.8500,0.75,5.800000e-01,0.240,-0.3700,-0.420000,0.690000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,10002,-18.0,18.0,433.22,469.42,-73.48,-12.270000,-12.270000,-11.936667,-11.770,...,-2.4900,-2.4625,-2.6125,-2.35,0.000000e+00,0.050,0.1000,-0.125000,-0.062500,0
1496,10003,-18.0,18.0,433.17,469.37,-73.53,-12.320000,-11.986667,-11.820000,-12.120,...,-2.5125,-2.6625,-2.4000,-0.05,-9.094947e-13,0.050,-0.1750,-0.112500,-0.166667,0
1497,10004,-18.0,18.0,433.22,469.42,-73.48,-12.270000,-11.936667,-11.770000,-12.070,...,-2.4625,-2.6125,-2.3500,0.00,5.000000e-02,0.100,-0.1250,-0.062500,-0.116667,0
1498,10005,-18.0,18.0,433.27,469.47,-73.43,-12.220000,-11.886667,-11.720000,-12.020,...,-2.4125,-2.5625,-2.3000,0.05,1.000000e-01,0.150,-0.0750,-0.012500,-0.066667,0


In [10]:
data=','.join([str(x) for x in evaluation_data_df.loc[0].tolist()[0:605]])
linear_predictor.predict(data)

{'predictions': [{'score': 0.6065435409545898, 'predicted_label': 0.0}]}

In [11]:
def eval_func(row):
    csv=','.join(row.tolist()[0:605])
    result=linear_predictor.predict(csv)
    return result
results=evaluation_data_df.astype(str).apply(eval_func,axis=1)

In [12]:
results[0]

{'predictions': [{'score': 0.6065435409545898, 'predicted_label': 0.0}]}

In [13]:
predicted_labels=[1 if i['predictions'][0]['predicted_label']==1.0 else 0 for i in results]

In [14]:
score=[i['predictions'][0]['score'] for i in results]

In [15]:
evaluation_data_df['predicted_labels']=predicted_labels
evaluation_data_df['score']=score
evaluation_data_df

,index,0,1,2,3,4,5,6,7,8,...,598,599,600,601,602,603,604,605,predicted_labels,score
0,8507,-18.0,18.0,422.10,458.30,-84.60,32.086667,32.063333,31.815000,31.840,...,0.0800,-0.77,-1.280000e+00,-0.770,0.6100,0.510000,0.340000,0,0,0.606544
1,8508,-18.0,18.0,422.71,458.91,-83.99,32.673333,32.425000,32.450000,32.560,...,-0.1600,-0.67,-1.600000e-01,1.220,1.1200,0.950000,0.610000,0,0,0.642264
2,8509,-18.0,18.0,422.76,458.96,-83.94,32.475000,32.500000,32.610000,32.460,...,-0.6200,-0.11,1.270000e+00,1.170,1.0000,0.660000,0.050000,0,0,0.654851
3,8510,-18.0,18.0,421.65,457.85,-85.05,31.390000,31.500000,31.350000,31.250,...,-1.2200,0.16,6.000000e-02,-0.110,-0.4500,-1.060000,-1.110000,0,0,0.623339
4,8511,-18.0,18.0,422.34,458.54,-84.36,32.190000,32.040000,31.940000,32.470,...,0.8500,0.75,5.800000e-01,0.240,-0.3700,-0.420000,0.690000,0,0,0.636813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,10002,-18.0,18.0,433.22,469.42,-73.48,-12.270000,-12.270000,-11.936667,-11.770,...,-2.6125,-2.35,0.000000e+00,0.050,0.1000,-0.125000,-0.062500,0,0,0.274829
1496,10003,-18.0,18.0,433.17,469.37,-73.53,-12.320000,-11.986667,-11.820000,-12.120,...,-2.4000,-0.05,-9.094947e-13,0.050,-0.1750,-0.112500,-0.166667,0,0,0.288146
1497,10004,-18.0,18.0,433.22,469.42,-73.48,-12.270000,-11.936667,-11.770000,-12.070,...,-2.3500,0.00,5.000000e-02,0.100,-0.1250,-0.062500,-0.116667,0,0,0.289661
1498,10005,-18.0,18.0,433.27,469.47,-73.43,-12.220000,-11.886667,-11.720000,-12.020,...,-2.3000,0.05,1.000000e-01,0.150,-0.0750,-0.012500,-0.066667,0,0,0.291181


In [16]:
evaluation_data_df[evaluation_data_df['predicted_labels']==1]

,index,0,1,2,3,4,5,6,7,8,...,598,599,600,601,602,603,604,605,predicted_labels,score
10,8517,-18.0,18.0,425.88,462.08,-80.82,34.1600,33.630,29.830,28.380000,...,4.2300,3.5400,3.25,1.88,1.650000,1.600000,0.920000,0,1,0.755484
99,8606,-18.0,18.0,423.26,459.46,-83.44,22.3325,22.210,22.210,22.300000,...,4.4000,4.2300,4.06,3.89,2.400000,1.870000,1.750000,0,1,0.756744
100,8607,-18.0,18.0,423.43,459.63,-83.27,22.5025,22.380,22.380,22.470000,...,4.5700,4.4000,4.23,4.06,2.570000,2.040000,1.920000,0,1,0.761357
101,8608,-18.0,18.0,423.26,459.46,-83.44,22.3325,22.210,22.210,22.300000,...,4.4000,4.2300,4.06,3.89,2.400000,1.870000,1.750000,0,1,0.756721
102,8609,-18.0,18.0,423.60,459.80,-83.10,22.6725,22.550,22.550,22.640000,...,4.7400,4.5700,4.40,4.23,2.740000,2.210000,2.090000,0,1,0.765899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,9495,-18.0,18.0,456.43,492.63,-50.27,35.3850,35.360,35.510,35.485000,...,-0.9225,-1.0075,-1.53,-1.95,-2.000000,-2.050000,-1.833333,1,1,0.793304
989,9496,-18.0,18.0,456.38,492.58,-50.32,35.3100,35.460,35.435,35.335000,...,-1.0575,-1.5800,-2.00,-2.05,-2.100000,-1.883333,-0.050000,1,1,0.778348
990,9497,-18.0,18.0,456.28,492.48,-50.42,35.2100,35.360,35.335,35.235000,...,-1.1575,-1.6800,-2.10,-2.15,-2.200000,-1.983333,-0.150000,1,1,0.775760
991,9498,-18.0,18.0,456.18,492.38,-50.52,35.1100,35.260,35.235,35.135000,...,-1.2575,-1.7800,-2.20,-2.25,-2.300000,-2.083333,-0.250000,1,1,0.773151


In [37]:
most_certain=evaluation_data_df.query('score>=0.91')
most_certain.sort_values('score',ascending=False)

,index,0,1,2,3,4,5,6,7,8,...,598,599,600,601,602,603,604,605,predicted_labels,score
724,9231,-18.0,18.0,461.02,497.22,-45.68,38.650,38.635,38.330000,36.89,...,7.7700,6.370,4.470000,3.830,3.660,3.130000,2.48,1,1,0.978773
725,9232,-18.0,18.0,460.99,497.19,-45.71,38.605,38.300,36.860000,36.32,...,6.3400,4.440,3.800000,3.630,3.100,2.450000,-0.03,1,1,0.976534
726,9233,-18.0,18.0,460.78,496.98,-45.92,38.395,38.090,36.650000,36.11,...,6.1300,4.230,3.590000,3.420,2.890,2.240000,-0.24,1,1,0.975806
731,9238,-18.0,18.0,460.19,496.39,-46.51,36.060,35.520,35.623333,35.52,...,3.0000,2.830,2.300000,1.650,-0.830,-0.573333,0.06,1,1,0.975150
727,9234,-18.0,18.0,460.52,496.72,-46.18,38.135,37.830,36.390000,35.85,...,5.8700,3.970,3.330000,3.160,2.630,1.980000,-0.50,1,1,0.974874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,9218,-18.0,18.0,453.28,489.48,-53.42,27.360,27.020,26.850000,26.72,...,-1.6000,-1.375,0.373333,1.555,0.010,0.240000,0.27,1,1,0.911364
649,9156,-18.0,18.0,458.74,494.94,-47.96,36.570,36.050,36.910000,37.78,...,0.6700,0.240,0.073333,-0.270,-0.130,0.185000,0.12,1,1,0.911160
710,9217,-18.0,18.0,453.26,489.46,-53.44,27.340,27.000,26.830000,26.70,...,-1.6200,-1.395,0.353333,1.535,-0.010,0.220000,0.25,1,1,0.911129
689,9196,-18.0,18.0,455.25,491.45,-51.45,34.400,34.200,33.650000,32.46,...,-0.2575,0.250,0.896667,1.130,1.160,0.880000,-0.22,1,1,0.910354


In [38]:
# Importing the metrics package from sklearn library
from sklearn import metrics
# Creating the confusion matrix
cm = metrics.confusion_matrix(most_certain[605], most_certain['predicted_labels'])
# Assigning columns names
cm_df = pd.DataFrame(cm, 
            columns = ['Predicted Negative', 'Predicted Positive'],
            index = ['Actual Negative', 'Actual Positive'])
# Showing the confusion matrix
cm_df

,Predicted Negative,Predicted Positive
Actual Negative,0,11
Actual Positive,0,139


In [ ]:
most_certain.query('predicted_labels==1').describe()

In [ ]:
most_certain.query('predicted_labels==0').describe()